<img src="ku_logo_uk_v.png" alt="drawing" width="500" style="float:right"/>


# Welcome to Workshop 11 - Introduction to Programming and Numerical Analysis !!!

# Plan for today
- Linear algebra
- Solving equations analytically
- Problem set 6

# Linear algebra

We use `SciPy's` module `linalg` to perform linear algebra operations: 

- Determinant, invert, norm.
- Solve a system of equations.
- Find eigenvalues.
- Etc.

[Module describtion](https://docs.scipy.org/doc/scipy/reference/linalg.html)

# Example:
$$ Ax = B $$


In [1]:
import numpy as np # import numpy
from scipy import linalg # import linalg

np.random.seed(666) # set seed

A = np.random.normal(size=(5,5)) # draw random A matrix
B = np.random.uniform(size=5) # draw random B vector

print(f'Matrix A: \n{A} \n\n Matrix b:\n{B}')

Matrix A: 
[[ 0.82418808  0.479966    1.17346801  0.90904807 -0.57172145]
 [-0.10949727  0.01902826 -0.94376106  0.64057315 -0.78644317]
 [ 0.60886999 -0.93101185  0.97822225 -0.73691806 -0.29873262]
 [-0.46058737 -1.08879299 -0.57577075 -1.68290077  0.22918525]
 [-1.75662522  0.84463262  0.27721986  0.85290153  0.1945996 ]] 

 Matrix b:
[0.41671201 0.74304719 0.36959638 0.07516654 0.77519298]


We use `lu_factor(a[, overwrite_a, check_finite])`, to compute pivoted LU decomposition of A matrix.
Decomposes A into upper and lower triangular matrix and solve through substitution.

And `lu_solve(lu_and_piv, b[, trans, …])`, to solve an equation system, $A x = B$, given the LU factorization of A.

In [2]:
LU, piv = linalg.lu_factor(A) # compute LU decomposition of A
x = linalg.lu_solve((LU,piv),B) # solve equation system
print(f'x is: {x}')

x is: [-0.61056339  0.02808207  0.26229122 -0.16346625 -1.30703677]


Or the simple way...
`solve(a, b[, sym_pos, lower, overwrite_a, …])`, solves the linear equation set $A x = B$ for the unknown x for square A matrix.

In [3]:
# Simple solver
x = linalg.solve(A,B)
print(f'x is: {x}')

x is: [-0.61056339  0.02808207  0.26229122 -0.16346625 -1.30703677]


# Solving Equations analytically
We use `Sympy` to work with formulas and model to enable us to translate analytics into python code - Wooow! 


Consider a utility function from a standard OLG model. 

Economic agents lives two periods (young/old) and obtains utility from consumption in both periods:

$$U_{i,t} = u_i(c_{i,1}) + \frac{1}{1+\rho}u_i(C_{i,2})$$

For simplicity we use log-utility.

In [4]:
import sympy as sm # import sympy

c1,c2 = sm.symbols("C_i1"), sm.symbols("C_i2") # define C_i1 and C_i2
rho = sm.symbols("rho") # define rho

#log-utility
uc1 = sm.ln(c1) 
uc2 = sm.ln(c2)
# Define U_it
U = uc1+1/(1+rho)*uc2
U


log(C_i1) + log(C_i2)/(rho + 1)

Let's take the derivative of $U$ with respect to $C_{i,2}$

In [5]:
sm.diff(U,c2)

1/(C_i2*(rho + 1))

## Sympy can turn your formulas into python functions.
Use the `Lambdify` method which takes a function and an iterable as argument. In our case the function is utility and the iterable is consumption: 

In [6]:
# define utility as a lamba function
util = sm.lambdify((c1,c2,rho),U)

print(f'Utility for C1=10, C2=5 and rho=0.1: {util(10,5,0.1):.2f}')



Utility for C1=10, C2=5 and rho=0.1: 3.77


# Let's try to optimize this 

In [7]:
from scipy import optimize

def obj(x): 
    if x[0] + x[1] > 10:  # Constraint, here if total consumption is above 10, utility is minus infinity. Could be a budget constraint maybe.
        return -np.inf
    else:
        return - util(x[0], x[1], 0.1) # negative utility, because we want to optimize. Note that rho = 0.1
opt = optimize.minimize(obj, #function
                        x0 = [2,2], # Guess
                        method = 'Nelder-Mead' #Method
                        ) 

c:\Users\45238\anaconda3\lib\site-packages\scipy\optimize\_optimize.py:790: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


In [8]:
print(f'''
    Optimal consumption in t=1: {round(opt.x[0], 2)}.
    Optimal consumption in t=2: {round(opt.x[1],2)}.
    With utility: {round(util(opt.x[0], opt.x[1], 0.1),2)}
    ''')


    Optimal consumption in t=1: 7.45.
    Optimal consumption in t=2: 3.27.
    With utility: 3.09
    


# Problem set 6